# Workshop: Train a Medical AI Model in One Day
## Notebook 3: Visualizing 2 Model's results in fiftyone

Load local predictions

In [ ]:
%pip install fiftyone

In [ ]:
import fiftyone as fo

In [2]:
# 🔧 Configuration
datasetUUID = "068818885c547b1e80002649e95254ce"
fiftyoneExportPath = "/Users/paularamos/Documents/GitHub/eyepop_ai/Voxel51-Medical-Collab/notebooks/.cache/voxel51/068818885c547b1e80002649e95254ce/fiftyone_dataset"

In [16]:
name = f"eyepop_dataset_{datasetUUID}"
# Check if the dataset exists
if name in fo.list_datasets():
    print(f"Dataset '{name}' exists. Loading...")
    dataset = fo.load_dataset(name)
else:
    print(f"Dataset '{name}' does not exist. Creating a new one...")
    # Clone the dataset with a new name and make it persistent
    dataset = fo.Dataset.from_dir(
        dataset_dir=fiftyoneExportPath,
        dataset_type=fo.types.FiftyOneDataset,
        name=name,
        persistent=True
    )
      
async def run():
    
    
    print("Loading dataset into FiftyOne...")    
    session = fo.launch_app(dataset, browser=True, auto=False)    
    
await run()

Dataset 'eyepop_dataset_068818885c547b1e80002649e95254ce' exists. Loading...
Loading dataset into FiftyOne...
Session launched. Run `session.show()` to open the App in a cell output.


In [17]:
# Iterate through the samples and update labels
for sample in dataset:
    for detection in sample.ground_truth.detections:
        if detection.label.startswith("stenosis"):
            detection.label = "stenosis"
    sample.save()

print("Labels updated successfully!")

Labels updated successfully!


In [18]:
# Grab a sample (first, specific index, or with a filter)
sample = dataset.first()  # or sample = dataset[42]

# Check if predictions_300 exists and print its structure
if sample.predictions_300 is not None:
    print("predictions_300 detections:")
    for det in sample.predictions_300.detections:
        print(det.label, det.bounding_box, det.confidence)
else:
    print("predictions_300 is None")

# Do the same for predictions_3k
if sample.predictions_3k is not None:
    print("\npredictions_3k detections:")
    for det in sample.predictions_3k.detections:
        print(det.label, det.bounding_box, det.confidence)
else:
    print("predictions_3k is None")

predictions_300 detections:
stenosis [0.19298240542411804, 0.39101943373680115, 0.08861394226551056, 0.17720861732959747] 0.8234073519706726
stenosis [0.1807192862033844, 0.34652823209762573, 0.1472143679857254, 0.2631445825099945] 0.6109607219696045

predictions_3k detections:
stenosis [0.187, 0.358, 0.09, 0.245] 0.782
stenosis [0.193, 0.411, 0.06, 0.154] 0.741


In [19]:
# Iterate over samples and fix labels
for sample in dataset:
    predictions = sample.predictions_300
    if predictions is not None and predictions.detections is not None:
        for detection in predictions.detections:
            if detection.label.startswith("stenosis"):
                detection.label = "stenosis"
        sample.predictions_300 = predictions  # assign back if modified
        sample.save()

In [ ]:
# Iterate over samples and fix labels
for sample in dataset:
    predictions = sample.predictions_3k
    if predictions is not None and predictions.detections is not None:
        for detection in predictions.detections:
            if detection.label.startswith("stenosis"):
                detection.label = "stenosis"
        sample.predictions_3k = predictions  # assign back if modified
        sample.save()

In [20]:
print(dataset)

Name:        eyepop_dataset_068818885c547b1e80002649e95254ce
Media type:  image
Num samples: 121
Persistent:  True
Tags:        []
Sample fields:
    id:                      fiftyone.core.fields.ObjectIdField
    filepath:                fiftyone.core.fields.StringField
    tags:                    fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:              fiftyone.core.fields.DateTimeField
    last_modified_at:        fiftyone.core.fields.DateTimeField
    partition:               fiftyone.core.fields.StringField
    uuid:                    fiftyone.core.fields.StringField
    ground_truth:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions_300:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    predictions_3k:          fiftyone.core.fields.EmbeddedDocu

In [21]:
results = dataset.evaluate_detections(
    "predictions_300",
    gt_field="ground_truth",
    eval_key="predictions_300_eval",
    compute_mAP=True,  # Avoids needing confidence
)
#results.print_report()

Evaluating detections...
 100% |█████████████████| 121/121 [145.9ms elapsed, 0s remaining, 829.6 samples/s]     
Performing IoU sweep...
 100% |█████████████████| 121/121 [123.6ms elapsed, 0s remaining, 979.2 samples/s]    


In [22]:
results2 = dataset.evaluate_detections(
    "predictions_3k",
    gt_field="ground_truth",
    eval_key="predictions_3k_eval",
    compute_mAP=True,  # Avoids needing confidence
)
#results.print_report()

Evaluating detections...
 100% |█████████████████| 121/121 [264.3ms elapsed, 0s remaining, 457.8 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 121/121 [262.0ms elapsed, 0s remaining, 461.8 samples/s]      


In [23]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)
results2.print_report(classes=classes_top10)

              precision    recall  f1-score   support

    stenosis       0.33      0.00      0.01       202

   micro avg       0.33      0.00      0.01       202
   macro avg       0.33      0.00      0.01       202
weighted avg       0.33      0.00      0.01       202

              precision    recall  f1-score   support

    stenosis       0.50      0.00      0.01       202

   micro avg       0.50      0.00      0.01       202
   macro avg       0.50      0.00      0.01       202
weighted avg       0.50      0.00      0.01       202



In [24]:
print(results.mAP())
print(results2.mAP())

0.0019801980198019802
0.0034653465346534654


In [25]:
import webbrowser

webbrowser.open("http://localhost:5151/")

True